# Анализ конверcии продаж

Датасет основан на данных рекламной кампании анонимной организации XYZ в социальной сети Facebook, взят с сайта [kaggle.com]( http://kaggle.com).

__ad_id:__ уникальный идентификатор для каждого объявления.

__xyzcampaignid:__ идентификатор, рекламной кампаниии XYZ.

__fbcampaignid:__ идентификатор, по которому Facebook отслеживает рекламные кампании.

__age:__ возраст человека, которому показано объявление.

__gender:__ пол человека, которому показано объявление.

__interest:__ код, определяющий категорию, к которой принадлежит интерес человека (интересы указаны в общедоступном профиле человека в Facebook).

__Impressions:__ сколько раз было показано объявление.

__Clicks:__ количество кликов по этому объявлению.

__Spent:__ сумма, которую компания XYZ заплатила Facebook за показ этой рекламы.

__Conversion:__ общее количество людей, купивших продукт после просмотра рекламы. (стоимость товара 5$)

__Задачи:__
- Расчитать Click-through-rate (CTR), Conversion Rate (CR), Cost Per Click (CPC), Cost Per Conversion, Return on Advertising Spend (ROAS). 
- После какой рекламной кампании продажи были выше?
- Какая кампания показала лучшие результаты? (По метрикам)
- У какого сегмента больше кликов? (Возраст, пол, интересы)
- Какой сегмент показал лучшие результаты? (По метрикам)








In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sales = pd.read_csv('/Users/User/Desktop/Projects/KAG_conversion_data.csv') 

## Подготовка данных

In [3]:
sales['Conversion'] = sales['Total_Conversion'] + sales['Approved_Conversion']
sales = sales.drop(['Total_Conversion','Approved_Conversion'],axis=1)

In [4]:
#Приведем названия к строчному виду
sales = sales.rename(columns=lambda x:x.lower())

In [5]:
sales.head()

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,impressions,clicks,spent,conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,3
1,708749,916,103917,30-34,M,16,17861,2,1.82,2
2,708771,916,103920,30-34,M,20,693,0,0.00,1
3,708815,916,103928,30-34,M,28,4259,1,1.25,1
4,708818,916,103928,30-34,M,28,4133,1,1.29,2


In [6]:
sales.info()#Нет потеряеных значений

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ad_id            1143 non-null   int64  
 1   xyz_campaign_id  1143 non-null   int64  
 2   fb_campaign_id   1143 non-null   int64  
 3   age              1143 non-null   object 
 4   gender           1143 non-null   object 
 5   interest         1143 non-null   int64  
 6   impressions      1143 non-null   int64  
 7   clicks           1143 non-null   int64  
 8   spent            1143 non-null   float64
 9   conversion       1143 non-null   int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 89.4+ KB


In [7]:
sales.age.value_counts()#значения в столбце age

30-34    426
45-49    259
35-39    248
40-44    210
Name: age, dtype: int64

In [8]:
#Переведем промежутки и пол к целочисленному значению
sales['age'] = sales['age'].map({'30-34':32,'45-49':47,'35-39':37,'40-44':42})
sales['gender'] = sales['gender'].map({'M':0,'F':1})

In [9]:
sales.head()#Значения поменялись

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,impressions,clicks,spent,conversion
0,708746,916,103916,32,0,15,7350,1,1.43,3
1,708749,916,103917,32,0,16,17861,2,1.82,2
2,708771,916,103920,32,0,20,693,0,0.00,1
3,708815,916,103928,32,0,28,4259,1,1.25,1
4,708818,916,103928,32,0,28,4133,1,1.29,2


## Расчет CTR, CR, CPC, CPC, ROAS

In [182]:
sales['ctr'] = round((sales['clicks']/sales['impressions'])*100, 4)

sales['cr'] = round(sales['conversion']/sales['impressions']*100, 3)

sales['cpc'] = round(sales['spent']/sales['clicks'], 2)

sales['cp_conversion'] = round(sales['spent']/sales['conversion'], 2)

sales['roas'] = round((sales['conversion']*5)/sales['spent'], 2)

In [183]:
# уберем значения inf
sales = sales.replace([np.inf, -np.inf], np.nan)

In [184]:
sales.head()

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,impressions,clicks,spent,conversion,ctr,cr,cpc,cp_conversion,roas
0,708746,916,103916,32,0,15,7350,1,1.43,3,0.0136,0.041,1.43,0.48,10.49
1,708749,916,103917,32,0,16,17861,2,1.82,2,0.0112,0.011,0.91,0.91,5.49
2,708771,916,103920,32,0,20,693,0,0.00,1,0.0000,0.144,NaN,0.00,NaN
3,708815,916,103928,32,0,28,4259,1,1.25,1,0.0235,0.023,1.25,1.25,4.00
4,708818,916,103928,32,0,28,4133,1,1.29,2,0.0242,0.048,1.29,0.64,7.75


## После какой рекламной кампании продажи были выше?

In [44]:
sales.groupby('xyz_campaign_id').agg({'conversion':'sum'}).sort_values(by='conversion',ascending=False)

,conversion
xyz_campaign_id,
1178,3541
936,720
916,82


После кампании __ID"1178"__ продажи были самыми высокими, а именно __3 541__ покупок или __17 705$__ в денежном эквиваленте.

## Какая кампания показала лучшие результаты? (По метрикам)

In [187]:
sales_agg = sales.groupby('xyz_campaign_id')\
                .agg(['sum'])[['interest','impressions','clicks','spent','conversion']]

In [188]:
sales_agg['ctr'] = round((sales_agg['clicks']/sales_agg['impressions'])*100, 4)

sales_agg['cr'] = round(sales_agg['conversion']/sales_agg['impressions']*100, 3)

sales_agg['cpc'] = round(sales_agg['spent']/sales_agg['clicks'], 2)

sales_agg['cp_conversion'] = round(sales_agg['spent']/sales_agg['conversion'], 2)

sales_agg['roas'] = round((sales_agg['conversion']*5)/sales_agg['spent'], 2)

In [189]:
sales_agg

,interest,impressions,clicks,spent,conversion,ctr,cr,cpc,cp_conversion,roas
,sum,sum,sum,sum,sum,,,,,
xyz_campaign_id,,,,,,,,,,
916,1187,482925,113,149.710001,82,0.0234,0.017,1.32,1.83,2.74
936,11622,8128187,1984,2893.369999,720,0.0244,0.009,1.46,4.02,1.24
1178,24643,204823716,36068,55662.149959,3541,0.0176,0.002,1.54,15.72,0.32


In [231]:
def best_result_camp (metric):
    if metric == 'cpc' or metric == 'cp_conversion':
        best = sales_agg.iloc[[sales_agg[metric].argmin()]][[metric]]
        return best
    else:
        best = sales_agg.iloc[[sales_agg[metric].argmax()]][[metric]]
        return best

Лучшая кампания по Click-through-rate 

In [232]:
best_result_camp('ctr')

,ctr
,
xyz_campaign_id,
936,0.0244


Лучшая кампания по Conversion Rate 

In [233]:
best_result_camp('cr')

,cr
,
xyz_campaign_id,
916,0.017


Лучшая кампания по Cost Per Click 

In [234]:
best_result_camp('cpc')

,cpc
,
xyz_campaign_id,
916,1.32


Лучшая кампания по Cost Per Conversion

In [235]:
best_result_camp('cp_conversion')

,cp_conversion
,
xyz_campaign_id,
916,1.83


Лучшая кампания по Return on Advertising Spend 

In [236]:
best_result_camp('roas')

,roas
,
xyz_campaign_id,
916,2.74


Кампании __"916"__ и __"936"__ можно признать успешными, так как у этих кампании  ROAS больше 1.

У __"916"__ показатель ROAS наибольший, однако конверсия ниже в несколько раз чем у __"936"__. 

Кампания __"1178"__ неудачная,даже несмотря на то у нее была самая высокая конверсия,но ROAS ниже 1, стоимость за клик и конверсию выше чем у других кампаний.

## У какого сегмента больше кликов?

In [134]:
sales.loc[[sales['clicks'].argmax()]][['gender','age','interest','clicks']]\
    .replace({32:'30-34',47:'45-49',37:'35-39',42:'40-44',0:'M',1:'F'})

,gender,age,interest,clicks
860,F,30-34,27,421


## Какой сегмент показал лучшие результаты? (По метрикам)

In [120]:
def best_result_seg (metric):
    best = sales.loc[[sales[metric].argmax()]][['gender','age','interest',metric]]
    best = best.replace({32:'30-34',47:'45-49',37:'35-39',42:'40-44',0:'M',1:'F'})
    return best

Лучший сегмент по Click-through-rate

In [121]:
best_result_seg('ctr')

,gender,age,interest,ctr
150,F,45-49,24,0.1059


Лучший сегмент по Conversion Rate

In [122]:
best_result_seg('cr')

,gender,age,interest,cr
464,M,30-34,24,0.02
